In [1]:
from ultralytics import YOLO
import cv2

In [2]:
model = YOLO("fkg_model.pt")

In [10]:
cap = cv2.VideoCapture(1)  #SET ACCOURDING TO YOUR WEBCAM
last_frames = [set()]
try:
    while cap.isOpened():
        _ , frame = cap.read()
        results = model.predict(frame,iou=0.3,verbose=False)
        box = results[0].boxes
        
        persist = last_frames[0].intersection(*last_frames)
        
        for b,l in zip(box.xyxy, box.cls):
            if l.item() not in persist:
                continue
            label = results[0].names[l.item()]
            boundary = b.int().tolist()
            frame = cv2.rectangle(frame, boundary[:2], boundary[2:], (0, 255, 0), 2)
            frame = cv2.putText(frame, label, boundary[:2], cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2, cv2.LINE_AA)
                
                
        if len(last_frames) > 2:
            last_frames.pop(0)
        last_frames.append(set(box.cls.tolist()))

        cv2.imshow("frame", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
finally:
    cap.release()
    cv2.destroyAllWindows()